In [115]:
import pandas as pd
from os import listdir
import unidecode

In [116]:
additional_info = pd.read_csv('additional_csv/addinfo.csv')

In [117]:
# Remove code, accencts and all names to lower case
additional_info['poblacion'] = additional_info['poblacion'].apply(lambda x: unidecode.unidecode(' '.join(x.split(' ')[1:])).lower())

In [191]:
all_files = []
path = 'province_houses/'
for filename in listdir(path):
    if 'houses' in filename and 'csv' in filename:
        municipio = unidecode.unidecode(filename.split('_')[1].split('.')[0])
        
        valor_empresa_muni = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains(municipio)) & (additional_info['source'] == 'Empresas')]['valor'].values[0].replace('.', ''))
        valor_poblacion_muni = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains(municipio)) & (additional_info['source'] == 'Poblacion')]['valor'].values[0].replace('.', ''))
        
        valor_empresa_nacional = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains('nacional')) & (additional_info['source'] == 'Empresas')]['valor'].values[0].replace('.', ''))
        valor_poblacion_nacional = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains('nacional')) & (additional_info['source'] == 'Poblacion')]['valor'].values[0].replace('.', ''))

        
        df = pd.read_csv(path + filename)
        df['source'] = filename.split('.')[0]
        df['number_of_companies'] = valor_empresa_muni
        df['muni_comanies_vs_national \%'] = ((valor_empresa_muni / valor_empresa_nacional) * 100).round(2)
        df['muni_comanies_vs_national \%'] = ((valor_empresa_muni / valor_empresa_nacional) * 100).round(2)

        df['population'] = valor_poblacion_muni
        
        all_files.append(df)

In [192]:
result = pd.concat(all_files)

In [176]:
result.to_csv('houses.csv')

In [52]:
# TODO: add rent & other attr. dataset

In [193]:
result

,ad_description,ad_last_update,air_conditioner,balcony,bath_num,built_in_wardrobe,chimney,condition,construct_date,energetic_certif,...,reduced_mobility,room_num,storage_room,swimming_pool,terrace,unfurnished,source,number_of_companies,muni_comanies_vs_national,population
0,EXCLSUIVA Fantástica casa construcción de lujo...,Anuncio actualizado el 1 de enero,0,1,4,1,0,segunda mano/buen estado,NaN,en trámite,...,0,3,1,1,0,NaN,houses_bizkaia,75628,2.27,1149628
1,NaN,más de 5 meses sin actualizar,0,1,3,0,0,segunda mano/para reformar,NaN,no indicado,...,0,8,0,0,0,NaN,houses_bizkaia,75628,2.27,1149628
2,NaN,más de 5 meses sin actualizar,0,0,1,0,0,segunda mano/para reformar,NaN,no indicado,...,0,3,0,0,0,NaN,houses_bizkaia,75628,2.27,1149628
3,piso situado en una calle peatonal. trastero 6...,más de 5 meses sin actualizar,0,0,1,0,0,segunda mano/buen estado,NaN,NaN,...,0,2,1,0,0,NaN,houses_bizkaia,75628,2.27,1149628
4,Se vente Casa Rústica con terreno en Karrantza...,más de 5 meses sin actualizar,0,0,sin baños,0,0,segunda mano/para reformar,NaN,no indicado,...,0,sin habitación,0,0,0,NaN,houses_bizkaia,75628,2.27,1149628
5,"Casa completamente a reformar.,No tiene terren...",más de 5 meses sin actualizar,0,0,3,0,0,segunda mano/para reformar,NaN,no indicado,...,0,5,0,0,0,NaN,houses_bizkaia,75628,2.27,1149628
6,Caserio situado a 15 km de bilbao. Consta de 2...,más de 5 meses sin actualizar,0,0,sin baños,0,1,segunda mano/para reformar,NaN,no indicado,...,0,sin habitación,1,0,0,NaN,houses_bizkaia,75628,2.27,1149628
7,NaN,más de 5 meses sin actualizar,0,0,2,1,0,segunda mano/buen estado,NaN,NaN,...,0,3,1,0,1,NaN,houses_bizkaia,75628,2.27,1149628
8,NaN,más de 5 meses sin actualizar,0,0,2,1,0,segunda mano/buen estado,NaN,en trámite,...,0,2,0,0,0,NaN,houses_bizkaia,75628,2.27,1149628
9,NaN,más de 5 meses sin actualizar,0,0,2,0,0,segunda mano/buen estado,NaN,no indicado,...,0,3,0,0,0,NaN,houses_bizkaia,75628,2.27,1149628


In [184]:
additional_info

,poblacion,source,valor
0,nacional,Empresas,3.337.646
1,albacete,Empresas,26.743
2,alicante/alacant,Empresas,138.964
3,almeria,Empresas,42.931
4,araba/alava,Empresas,19.147
5,asturias,Empresas,68.688
6,avila,Empresas,10.170
7,badajoz,Empresas,41.490
8,"balears, illes",Empresas,96.638
9,barcelona,Empresas,468.777
